In [1]:
import pandas as pd

df = pd.read_csv("pokemon.csv")
gen = 3
df_gen = df[df["generation"] <= gen]
df_gen.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


In [2]:
df_gen["name"] = df_gen["name"].apply(lambda name: name.lower())
sorted(df_gen["name"].to_list())[:10]

C:\Users\durbi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['abra',
 'absol',
 'aerodactyl',
 'aggron',
 'aipom',
 'alakazam',
 'altaria',
 'ampharos',
 'anorith',
 'arbok']

In [3]:
class Type():
    attack_pros = []
    attack_cons = []
    attack_no_effect = []
    def_pros = []
    def_cons = []
    def_no_effect = []
    name = "???"
    

def matchup(type1 : Type, type2 : Type):
    type1_attack_score = 0
    type1_attack_score += len([
        ptype for ptype in type1.attack_pros if ptype == type2.name
    ])
    type1_attack_score -= len([
        ptype for ptype in type1.attack_cons if ptype == type2.name
    ])
    type1_attack_score -= 2.5*len([
        ptype for ptype in type1.attack_no_effect if ptype == type2.name
    ])

    type1_def_score = 0
    type1_def_score += len([
        ptype for ptype in type1.def_pros if ptype == type2.name
    ])
    type1_def_score -= len([
        ptype for ptype in type1.def_cons if ptype == type2.name
    ])
    type1_def_score += 2.5*len([
        ptype for ptype in type1.def_no_effect if ptype == type2.name
    ])

    return type1_attack_score, type1_def_score


    


In [4]:
ptype_df = pd.read_csv("pokemon_type_matchup.csv")
ptype_df.head()

,Type,Strong against,Weak against,Resistant to,Weak to
0,Normal,–,"Rock, Steel, Ghost (no effect)",Ghost (no effect),Fighting
1,Fighting,"Normal, Rock, Steel, Ice, Dark","Flying, Poison, Bug, Psychic, Fairy, Ghost (no...","Rock, Bug, Dark","Flying, Psychic, Fairy"
2,Flying,"Fighting, Bug, Grass","Rock, Steel, Electric","Fighting, Bug, Grass, Ground (no effect)","Rock, Electric, Ice"
3,Poison,"Grass, Fairy","Poison, Ground, Rock, Ghost, Steel (no effect)","Fighting, Poison, Grass, Fairy","Ground, Psychic"
4,Ground,"Poison, Rock, Steel, Fire, Electric","Bug, Grass, Flying (no effect)","Poison, Rock, Electric (no effect)","Water, Grass, Ice"


In [5]:
ptype_df.columns = "type strong_against weak_against resistant_to weak_to".split()
ptype_df.head()

,type,strong_against,weak_against,resistant_to,weak_to
0,Normal,–,"Rock, Steel, Ghost (no effect)",Ghost (no effect),Fighting
1,Fighting,"Normal, Rock, Steel, Ice, Dark","Flying, Poison, Bug, Psychic, Fairy, Ghost (no...","Rock, Bug, Dark","Flying, Psychic, Fairy"
2,Flying,"Fighting, Bug, Grass","Rock, Steel, Electric","Fighting, Bug, Grass, Ground (no effect)","Rock, Electric, Ice"
3,Poison,"Grass, Fairy","Poison, Ground, Rock, Ghost, Steel (no effect)","Fighting, Poison, Grass, Fairy","Ground, Psychic"
4,Ground,"Poison, Rock, Steel, Fire, Electric","Bug, Grass, Flying (no effect)","Poison, Rock, Electric (no effect)","Water, Grass, Ice"


In [6]:
def get_typelist(typedata):
    if "–" in typedata:
        return []
    typelist = typedata.split(", ")
    typelist = [s.lower() for s in typelist if (not "(no effect)" in s)]
    return typelist

def get_noeffectlist(typedata):
    noeffectlist = typedata.split(", ")
    noeffectlist = [s.lower() for s in noeffectlist if ("(no effect)" in s)]
    noeffectlist = [s.replace(" (no effect)", "") for s in noeffectlist]
    return noeffectlist

# Get no effect data
ptype_df["att_no_effect"] = ptype_df["weak_against"].apply(get_noeffectlist)
ptype_df["def_no_effect"] = ptype_df["resistant_to"].apply(get_noeffectlist)


# Format all columns
ptype_df["type"] = ptype_df["type"].apply(lambda s: s.lower())

for column_name in ptype_df.columns[1:-2]:
    if not (type(ptype_df[column_name].to_list()[0]) == list):
        ptype_df[column_name] = ptype_df[column_name].apply(get_typelist)

# Show head to make sure everything works
ptype_df.head()

,type,strong_against,weak_against,resistant_to,weak_to,att_no_effect,def_no_effect
0,normal,[],"[rock, steel]",[],[fighting],[ghost],[ghost]
1,fighting,"[normal, rock, steel, ice, dark]","[flying, poison, bug, psychic, fairy]","[rock, bug, dark]","[flying, psychic, fairy]",[ghost],[]
2,flying,"[fighting, bug, grass]","[rock, steel, electric]","[fighting, bug, grass]","[rock, electric, ice]",[],[ground]
3,poison,"[grass, fairy]","[poison, ground, rock, ghost]","[fighting, poison, grass, fairy]","[ground, psychic]",[steel],[]
4,ground,"[poison, rock, steel, fire, electric]","[bug, grass]","[poison, rock]","[water, grass, ice]",[flying],[electric]


In [7]:
ptype_df

,type,strong_against,weak_against,resistant_to,weak_to,att_no_effect,def_no_effect
0,normal,[],"[rock, steel]",[],[fighting],[ghost],[ghost]
1,fighting,"[normal, rock, steel, ice, dark]","[flying, poison, bug, psychic, fairy]","[rock, bug, dark]","[flying, psychic, fairy]",[ghost],[]
2,flying,"[fighting, bug, grass]","[rock, steel, electric]","[fighting, bug, grass]","[rock, electric, ice]",[],[ground]
3,poison,"[grass, fairy]","[poison, ground, rock, ghost]","[fighting, poison, grass, fairy]","[ground, psychic]",[steel],[]
4,ground,"[poison, rock, steel, fire, electric]","[bug, grass]","[poison, rock]","[water, grass, ice]",[flying],[electric]
5,rock,"[flying, bug, fire, ice]","[fighting, ground, steel]","[normal, flying, poison, fire]","[fighting, ground, steel, water, grass]",[],[]
6,bug,"[grass, psychic, dark]","[fighting, flying, poison, ghost, steel, fire,...","[fighting, ground, grass]","[flying, rock, fire]",[],[]
7,ghost,"[ghost, psychic]",[dark],"[poison, bug]","[ghost, dark]",[normal],"[normal, fighting]"
8,steel,"[rock, ice, fairy]","[steel, fire, water, electric]","[normal, flying, rock, bug, steel, grass, psyc...","[fighting, ground, fire]",[],[poison]
9,fire,"[bug, steel, grass, ice]","[rock, fire, water, dragon]","[bug, steel, fire, grass, ice]","[ground, rock, water]",[],[]


In [70]:
import numpy as np

def matchup(types1, types2):
    type1_attack_score = 0
    type1_def_score = 0
    
    for type1 in types1:
        if type1 is np.nan:
                continue
        type1_data = ptype_df[ptype_df["type"] == type1]
        for type2 in types2:
            if type2 is np.nan:
                continue

            #type2_data = ptype_df[ptype_df["type"] == type2]
            type1_attack_score += len([
                ptype for ptype in type1_data["strong_against"].values[0] if ptype == type2
            ])
            type1_attack_score -= len([
                ptype for ptype in type1_data["weak_against"].values[0]if ptype == type2
            ])
            type1_attack_score -= 1000*len([
                ptype for ptype in type1_data["att_no_effect"].values[0] if ptype == type2
            ])

            type1_def_score += len([
                ptype for ptype in type1_data["resistant_to"].values[0] if ptype == type2
            ])
            type1_def_score -= len([
                ptype for ptype in type1_data["weak_to"].values[0] if ptype == type2
            ])
            type1_def_score += 1000*len([
                ptype for ptype in type1_data["def_no_effect"].values[0] if ptype == type2
            ])
        #print(f"{type1} {type2} {type1_attack_score} {type1_def_score}")

    return type1_attack_score, type1_def_score
    

In [71]:
matchup(["dragon", np.nan], ["dragon", np.nan])

(1, -1)

In [10]:
df_gen.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


In [85]:
def get_standings(pokemon):
    row = df_gen["name"] == pokemon
    types = df_gen[row]["type1"].values[0], df_gen[row]["type2"].values[0]

    att_wins = 0
    beats_pokemon_att = lambda pokemon2: matchup(types, [df_gen[df_gen["name"] == pokemon2]["type1"].values[0], df_gen[df_gen["name"] == pokemon2]["type2"].values[0]])[0] > 0
    beats_pokemon_def = lambda pokemon2: matchup(types, [df_gen[df_gen["name"] == pokemon2]["type1"].values[0], df_gen[df_gen["name"] == pokemon2]["type2"].values[0]])[1] > 0
    return [sum(df_gen["name"].apply(beats_pokemon_att)), sum(df_gen["name"].apply(beats_pokemon_def))]


In [72]:
get_standings("charizard")
all_standings = df_gen["name"].apply(lambda name: (name, get_standings(name)))

In [81]:
sorted(all_standings.to_list(), key=lambda v: v[1][1])[:5]

[('magnemite', [79, 263]),
 ('magneton', [79, 263]),
 ('skarmory', [109, 258]),
 ('mawile', [87, 254]),
 ('registeel', [47, 242])]

In [88]:
def get_standings_optimized(pokemon):
    row = df_gen["name"] == pokemon
    types = df_gen[row]["type1"].values[0], df_gen[row]["type2"].values[0]


    att_wins = lambda matchup_entry: matchup_entry[0] > 0
    def_wins = lambda matchup_entry: matchup_entry[1] > 0

    att_draws = lambda matchup_entry: matchup_entry[0] == 0
    def_draws = lambda matchup_entry: matchup_entry[1] == 0

    att_losses = lambda matchup_entry: matchup_entry[0] < 0
    def_losses = lambda matchup_entry: matchup_entry[1] < 0
    
    matchups = df_gen["name"].apply(
        lambda pokemon2: (lambda pokemon2_entry: matchup(types, [pokemon2_entry["type1"].values[0], pokemon2_entry["type2"].values[0]]))(df_gen[df_gen["name"] == pokemon2])
    )
    return [sum(matchups.apply(scoring_function)) for scoring_function in [
        att_wins, att_draws, att_losses, def_wins, def_draws, def_losses
    ]]


In [89]:
all_standings_optimized = df_gen["name"].apply(lambda name: (name, get_standings_optimized(name)))

In [98]:
sorted(all_standings_optimized.to_list(), key=lambda v: -v[1][0])

data = all_standings_optimized.to_list()
columns = ["name", "att_wins", "att_draws", "att_losses", "def_wins", "def_draws", "def_losses"]

standings_df = pd.DataFrame([[x[0]] + x[1] for x in data], columns=columns) 
standings_df.head()

,name,att_wins,att_draws,att_losses,def_wins,def_draws,def_losses
0,bulbasaur,83,148,155,141,121,124
1,ivysaur,83,148,155,141,121,124
2,venusaur,83,148,155,141,121,124
3,charmander,88,176,122,107,182,97
4,charmeleon,88,176,122,107,182,97


In [101]:
standings_df.sort_values(by="def_losses", ascending=False)

,name,att_wins,att_draws,att_losses,def_wins,def_draws,def_losses
220,piloswine,87,170,129,50,179,157
219,swinub,87,170,129,50,179,157
27,sandslash,87,170,129,50,179,157
26,sandshrew,87,170,129,50,179,157
255,combusken,98,103,185,108,129,149
...,...,...,...,...,...,...,...
160,sentret,0,334,52,10,358,18
161,furret,0,334,52,10,358,18
351,kecleon,0,334,52,10,358,18
127,tauros,0,334,52,10,358,18
